In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go  # Add this line
import os
from IPython.display import display
import gspread
from datetime import datetime, timedelta, date, time

import holidays
from dateutil.easter import easter

In [14]:
def ler_gsheet_reservas_consolidadas(sheet_key, worksheet_name, credentials_path='credentials.json'):
    """
    Lê os dados de uma aba específica de uma Planilha Google.
    
    *** VERSÃO CORRIGIDA: Assume cabeçalhos na Linha 1 e dados na Linha 2 ***
    (Ideal para a aba 'Reservas Consolidadas')
    """
    print(f"\n--- Iniciando leitura da Planilha Google ---")
    print(f"  Planilha Key: {sheet_key}")
    print(f"  Aba         : {worksheet_name}")

    if not os.path.exists(credentials_path):
        print(f"ERRO: Arquivo de credenciais '{credentials_path}' não encontrado.")
        return None

    try:
        # 1. Autenticar
        print("Autenticando com conta de serviço...")
        gc = gspread.service_account(filename=credentials_path)
        print("Autenticação OK.")

        # 2. Abrir a Planilha
        print(f"Abrindo planilha...")
        sh = gc.open_by_key(sheet_key)
        print("Planilha aberta.")

        # 3. Selecionar a Aba
        try:
            worksheet = sh.worksheet(worksheet_name)
            print(f"Aba '{worksheet_name}' encontrada.")
        except gspread.exceptions.WorksheetNotFound:
            print(f"ERRO: Aba '{worksheet_name}' não encontrada na planilha.")
            return None

        # 4. Ler Todos os Valores
        print(f"Lendo todos os dados da aba '{worksheet_name}'...")
        all_values = worksheet.get_all_values(value_render_option='FORMATTED_VALUE')
        
        if not all_values or len(all_values) < 2:
             print("Aviso: Aba está vazia ou contém apenas cabeçalho.")
             return pd.DataFrame()
             
        headers = all_values[0] # Linha 1 (índice 0) como cabeçalho
        data = all_values[1:]   # Dados da Linha 2 (índice 1) em diante

        # Cria o DataFrame
        print("Criando DataFrame Pandas...")
        df = pd.DataFrame(data, columns=headers)
        print("DataFrame criado com sucesso.")
        return df

    except Exception as e:
        print(f"ERRO inesperado durante a leitura da Google Sheet: {e}")
        return None

In [15]:
def get_holidays(years=[2025,2026]):
    """
    Retorna um DataFrame com os feriados dos anos especificados.
    Args:
        years: Um ano (int) ou uma lista de anos (list).
    Colunas: "Feriado", "Data", "Dia da Semana", "Mês", "Abrangência", "Feriadão"
    Abrangência: "Brasil", "Pernambuco", "Recife"
    Feriadão: "Sim" para Carnaval, Semana Santa e feriados em Seg/Sex.
    """
    if isinstance(years, int):
        years = [years]
        
    all_holidays_list = []
    
    dia_semana_map = {
        0: "Segunda-feira",
        1: "Terça-feira",
        2: "Quarta-feira",
        3: "Quinta-feira",
        4: "Sexta-feira",
        5: "Sábado",
        6: "Domingo"
    }
    
    mes_map = {
        1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
        5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
        9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
    }

    for year in years:
        # 1. Feriados Nacionais (Brasil)
        br_holidays = holidays.Brazil(years=year)
        
        # 2. Feriados Estaduais (Pernambuco)
        pe_holidays = holidays.Brazil(subdiv='PE', years=year)
        
        # 3. Feriados Municipais (Recife) e Outros Manuais
        manual_holidays = {}
        
        # Recife Fixos
        manual_holidays[date(year, 3, 12)] = ("Aniversário do Recife", "Recife")
        manual_holidays[date(year, 6, 24)] = ("São João", "Recife")
        manual_holidays[date(year, 7, 16)] = ("Nossa Senhora do Carmo", "Recife")
        manual_holidays[date(year, 12, 8)] = ("Nossa Senhora da Conceição", "Recife")
        
        # Data Magna (Garantir 6 de Março)
        manual_holidays[date(year, 3, 6)] = ("Data Magna de Pernambuco", "Pernambuco")
        
        # Outros Feriados/Datas Comemorativas Solicitadas
        manual_holidays[date(year, 8, 11)] = ("Criação dos Cursos Jurídicos", "Brasil") 
        manual_holidays[date(year, 10, 15)] = ("Dia dos Professores", "Brasil")
        manual_holidays[date(year, 10, 28)] = ("Dia do Servidor Público", "Brasil")
        
        # Dia do Comerciário (3ª segunda-feira de outubro)
        oct_1 = date(year, 10, 1)
        first_monday_offset = (7 - oct_1.weekday()) % 7
        first_monday = oct_1 + timedelta(days=first_monday_offset)
        third_monday = first_monday + timedelta(weeks=2)
        manual_holidays[third_monday] = ("Dia do Comerciário", "Recife")

        # Datas Móveis (Baseadas na Páscoa)
        easter_date = easter(year)
        
        # Carnaval e Semana Santa
        # Sábado de Zé Pereira (Carnaval) = Páscoa - 50 dias
        carnaval_sat = easter_date - timedelta(days=50)
        manual_holidays[carnaval_sat] = ("Sábado de Carnaval", "Brasil")
        
        # Domingo de Carnaval = Páscoa - 49 dias
        carnaval_sun = easter_date - timedelta(days=49)
        manual_holidays[carnaval_sun] = ("Domingo de Carnaval", "Brasil")

        # Segunda de Carnaval = Páscoa - 48 dias
        carnaval_mon = easter_date - timedelta(days=48)
        manual_holidays[carnaval_mon] = ("Segunda-feira de Carnaval", "Brasil") 
        
        # Terça de Carnaval = Páscoa - 47 dias
        carnaval_tue = easter_date - timedelta(days=47)
        manual_holidays[carnaval_tue] = ("Terça-feira de Carnaval", "Brasil")
        
        # Quarta-feira de Cinzas = Páscoa - 46 dias
        cinzas = easter_date - timedelta(days=46)
        manual_holidays[cinzas] = ("Quarta-feira de Cinzas", "Brasil")
        
        # Quinta-feira Santa = Páscoa - 3 dias
        quinta_santa = easter_date - timedelta(days=3)
        manual_holidays[quinta_santa] = ("Quinta-feira Santa", "Recife")
        
        # Domingo de Páscoa
        manual_holidays[easter_date] = ("Domingo de Páscoa", "Brasil")
        
        # Corpus Christi (60 dias após a Páscoa)
        corpus_christi = easter_date + timedelta(days=60)
        manual_holidays[corpus_christi] = ("Corpus Christi", "Recife")
        
        # Definir períodos de feriadão fixos (Carnaval e Semana Santa)
        carnaval_dates = {carnaval_sat, carnaval_sun, carnaval_mon, carnaval_tue, cinzas}
        
        # Semana Santa: Quinta até Domingo (Sexta já é feriado, Domingo é Páscoa)
        sexta_santa = easter_date - timedelta(days=2)
        semana_santa_dates = {quinta_santa, sexta_santa, easter_date}
        
        # Unir todas as datas
        all_dates = set(br_holidays.keys()) | set(pe_holidays.keys()) | set(manual_holidays.keys())
        
        sorted_dates = sorted(list(all_dates))
        
        for feriado_date in sorted_dates:
            name = ""
            scope = ""
            
            # Verificar manuais primeiro para garantir override
            if feriado_date in manual_holidays:
                name, scope = manual_holidays[feriado_date]
            else:
                is_national = feriado_date in br_holidays
                is_state = feriado_date in pe_holidays and not is_national
                
                if is_national:
                    scope = "Brasil"
                    name = br_holidays.get(feriado_date)
                elif is_state:
                    scope = "Pernambuco"
                    name = pe_holidays.get(feriado_date)
                else:
                    continue
            
            # Filtrar "Revolução Pernambucana" se não for 6 de março (Data Magna)
            if name == "Revolução Pernambucana" and feriado_date != date(year, 3, 6):
                continue
                
            # Filtrar "Carnaval" genérico da lib holidays se já temos os específicos
            if name == "Carnaval" and feriado_date in manual_holidays:
                 pass
            
            # Filtrar "Quarta-feira de Cinzas" genérico se já temos
            if name == "Quarta-feira de Cinzas" and feriado_date in manual_holidays:
                 pass

            # Lógica de Feriadão
            is_feriadao = "Não"
            weekday = feriado_date.weekday()
            
            # Carnaval ou Semana Santa
            if feriado_date in carnaval_dates or feriado_date in semana_santa_dates:
                is_feriadao = "Sim"
            # Segunda (0) ou Sexta (4)
            elif weekday == 0 or weekday == 4:
                is_feriadao = "Sim"

            all_holidays_list.append({
                "Feriado": name,
                "Data": feriado_date.strftime("%d/%m/%Y"),
                "Dia da Semana": dia_semana_map[weekday],
                "Mês": mes_map[feriado_date.month],
                "Abrangência": scope,
                "Feriadão": is_feriadao,
                "_date_obj": feriado_date
            })
        
    df = pd.DataFrame(all_holidays_list)
    if "_date_obj" in df.columns:
        df = df.drop(columns=["_date_obj"])
        
    return df

In [16]:
get_holidays([2025,2026,2027])

,Feriado,Data,Dia da Semana,Mês,Abrangência,Feriadão
0,Confraternização Universal,01/01/2025,Quarta-feira,Janeiro,Brasil,Não
1,Sábado de Carnaval,01/03/2025,Sábado,Março,Brasil,Sim
2,Domingo de Carnaval,02/03/2025,Domingo,Março,Brasil,Sim
3,Segunda-feira de Carnaval,03/03/2025,Segunda-feira,Março,Brasil,Sim
4,Terça-feira de Carnaval,04/03/2025,Terça-feira,Março,Brasil,Sim
...,...,...,...,...,...,...
76,Finados,02/11/2027,Terça-feira,Novembro,Brasil,Não
77,Proclamação da República,15/11/2027,Segunda-feira,Novembro,Brasil,Sim
78,Dia Nacional de Zumbi e da Consciência Negra,20/11/2027,Sábado,Novembro,Brasil,Não
79,Nossa Senhora da Conceição,08/12/2027,Quarta-feira,Dezembro,Recife,Não


In [17]:
def criar_grafico_ocupacao(df_grafico, exibir=True): # <-- NOVO PARÂMETRO
    """
    (Versão Final - Com controle de exibição)
    Adicionado parâmetro 'exibir' para permitir edição do gráfico antes de mostrar.
    """
    
    if df_grafico is None or df_grafico.empty:
        print("DataFrame vazio. Não é possível gerar o gráfico.")
        return None

    # ... (Todo o código de preparação de dados permanece IGUAL) ...
    # --- Definições de Data ---
    hoje = pd.to_datetime('today').normalize()
    agora = pd.to_datetime('now')
    
    zoom_inicio = hoje - pd.Timedelta(days=10)
    zoom_fim = hoje + pd.Timedelta(days=20)

    if exibir: # Só imprime se for exibir agora
        print("Criando o gráfico de ocupação...")

    # --- Cores ---
    colors = {
        'Booking': 'rgb(46, 137, 205)', 'Airbnb': 'rgb(255, 90, 95)',      
        'Direto': 'rgb(75, 181, 67)', 'Outro': 'rgb(255, 0, 0)'          
    }

    # --- Preparação dos Dados (Cópia do código anterior) ---
    try:
        df_grafico = df_grafico.copy() # Trabalha numa cópia para não afetar o original
        df_grafico['Início'] = pd.to_datetime(df_grafico['Início'], errors='coerce', dayfirst=True)
        df_grafico['Fim'] = pd.to_datetime(df_grafico['Fim'], errors='coerce', dayfirst=True)
        df_grafico = df_grafico.dropna(subset=['Início', 'Fim'])
    except Exception as e:
        print(f"ERRO de Data: {e}")
        return None

    meia_noite = time(0, 0, 0)
    condicao_inicio_00h = df_grafico['Início'].dt.time == meia_noite
    condicao_fim_00h = df_grafico['Fim'].dt.time == meia_noite
    df_grafico.loc[condicao_inicio_00h, 'Início'] += pd.to_timedelta('16 hours')
    df_grafico.loc[condicao_fim_00h, 'Fim'] += pd.to_timedelta('11 hours')

    df_grafico = df_grafico[df_grafico['Fim'] >= hoje]
    if df_grafico.empty: return None

    df_grafico['Texto_Barra'] = (
        df_grafico['Início'].dt.day.astype(str) + '-' + df_grafico['Fim'].dt.day.astype(str)
    )
    ultima_data_reserva = df_grafico['Fim'].max()
    num_apartamentos = len(df_grafico['Apartamento'].unique())
    altura_dinamica = 300 + (num_apartamentos * 50)

    # --- CRIAÇÃO DO GRÁFICO ---
    fig = px.timeline(df_grafico, x_start="Início", x_end="Fim", y="Apartamento",
                      color="Origem", title="Mapa de Ocupação", color_discrete_map=colors,
                      text="Texto_Barra")
    
    fig.update_traces(textposition='inside', textfont=dict(color='black', size=11))

    # --- FUNDO ALTERNADO ---
    data_inicio_fundo = zoom_inicio
    data_fim_fundo = max(zoom_fim, ultima_data_reserva) + pd.Timedelta(days=5)
    dias_totais = (data_fim_fundo - data_inicio_fundo).days + 1
    
    COR_FERIADO, COR_DOMINGO, COR_SABADO, COR_DIA_UTIL = "#FFEBEE", "#E0E0E0", "#F5F5F5", "white"
    set_datas_feriados = set()
    try:
        anos = sorted(list(set(df_grafico['Início'].dt.year.tolist() + df_grafico['Fim'].dt.year.tolist())))
        if anos:
            df_f = get_holidays(anos)
            for _, r in df_f.iterrows():
                d = datetime.strptime(r['Data'], '%d/%m/%Y').date() if isinstance(r['Data'], str) else r['Data'].date()
                set_datas_feriados.add(d)
    except: pass

    for i in range(dias_totais):
        dia = data_inicio_fundo + pd.Timedelta(days=i)
        cor = COR_DIA_UTIL
        if dia.date() in set_datas_feriados: cor = COR_FERIADO
        elif dia.weekday() == 6: cor = COR_DOMINGO
        elif dia.weekday() == 5: cor = COR_SABADO
        
        if cor != "white":
            fig.add_shape(type="rect", x0=dia, y0=0, x1=dia + pd.Timedelta(days=1), y1=1,
                          xref="x", yref="paper", fillcolor=cor, layer="below", line_width=0)
        
        if dia.day == 1: # Mês
            nome_mes = dia.strftime('%B').capitalize()
            ano_mes = dia.strftime('%Y')
            fig.add_annotation(x=dia, y=1, yref="paper", text=f"<b>{nome_mes} {ano_mes}</b>",
                               showarrow=False, xanchor="left", yanchor="bottom", yshift=45, font=dict(color="black", size=14))
            fig.add_shape(type="line", x0=dia, y0=0, x1=dia, y1=1, xref="x", yref="paper", line=dict(color="black", width=1.5), opacity=0.3)

    # --- Traço Fantasma ---
    fig.add_trace(go.Scatter(x=[zoom_inicio, zoom_fim], y=[df_grafico['Apartamento'].iloc[0]]*2, 
                   mode='markers', xaxis='x2', opacity=0, showlegend=False, hoverinfo='skip'))

    # --- Linha Agora ---
    x_agora = agora.to_pydatetime()
    fig.add_shape(type="line", x0=x_agora, y0=0, x1=x_agora, y1=1, xref="x", yref="paper", line=dict(color="black", width=1.5, dash="dash"))
    fig.add_annotation(x=x_agora, y=0, yref="paper", text="Agora", showarrow=False, font=dict(color="black", size=10, weight="bold"), xanchor="right", yanchor="bottom")

    # --- Feriados (Texto) ---
    try:
        if anos:
            for _, r in df_f.iterrows():
                dt = datetime.strptime(r['Data'], '%d/%m/%Y') if isinstance(r['Data'], str) else r['Data']
                if zoom_inicio <= dt <= ultima_data_reserva:
                    dt12 = dt.replace(hour=12, minute=0, second=0)
                    fig.add_shape(type="line", x0=dt12, y0=0, x1=dt12, y1=1, xref="x", yref="paper", line=dict(color="gray", width=1, dash="dot"), opacity=0.4)
                    fig.add_annotation(x=dt12, y=1, yref="paper", text=r['Feriado'], showarrow=False, xanchor="left", yanchor="top", textangle=-90, font=dict(color="#555555", size=9))
    except: pass

    # --- Layout ---
    ordem_apartamentos = ['AP-101', 'AP-201', 'CBL004', 'SM-C108', 'SM-D014']
    fig.update_layout(
        title_text="Mapa de Reservas", height=altura_dinamica, xaxis_rangeslider_visible=True,
        yaxis_autorange='reversed', hovermode='x unified', barcornerradius=5, plot_bgcolor='white',
        margin=dict(t=150, b=40),
        xaxis=dict(title="", side="bottom", showgrid=True, gridcolor="#E5E5E5", showticklabels=False, dtick=86400000, range=[zoom_inicio, zoom_fim]),
        xaxis2=dict(title="", side="top", overlaying="x", showgrid=False, matches="x", tickformat='<b>%d</b><br>%a', tickangle=0, dtick=86400000, ticklabelmode="period"),
        yaxis=dict(title="", showgrid=True, gridcolor="#E5E5E5", categoryorder='array', categoryarray=ordem_apartamentos),
        legend=dict(orientation="h", yanchor="bottom", y=1.35, xanchor="right", x=1)
    )
    
    # --- MUDANÇA AQUI: Controle de exibição ---
    if exibir:
        print("Gráfico gerado.")
        fig.show()
        
    return fig

In [18]:
# --- Configuração ---
SHEET_KEY = '1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY'
CREDS_FILE = 'credentials2.json' # (ou o nome correto das suas credenciais)
# Define o nome do arquivo HTML de saída
html_output_file = 'ocupacao_apartamentos.html' # Nome de arquivo atualizado

df_para_grafico = ler_gsheet_reservas_consolidadas(SHEET_KEY, 'Reservas Consolidadas', CREDS_FILE)

display(df_para_grafico.head())  # Verifica os dados preparados

fig_ocupacao = criar_grafico_ocupacao(df_para_grafico)



# Salva o gráfico
fig_ocupacao.write_html(html_output_file)
print(f"Gráfico salvo como '{html_output_file}'.")



--- Iniciando leitura da Planilha Google ---
  Planilha Key: 1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY
  Aba         : Reservas Consolidadas
Autenticando com conta de serviço...
Autenticação OK.
Abrindo planilha...
Planilha aberta.
Aba 'Reservas Consolidadas' encontrada.
Lendo todos os dados da aba 'Reservas Consolidadas'...
Criando DataFrame Pandas...
DataFrame criado com sucesso.


,Início,Fim,Dias,Pessoas,Quem,Origem,Recebido,Despesas,A receber,Total Lq,...,Diária BT,Diária Lq,Com.,Diária BT PAX,Diária Lq PAX,Status,Data Reserva,,Email,Apartamento
0,20/06/2025 15:00,24/06/2025 11:00,4,5,Mariana,Direto,"1.800,00",,,"1.800,00",...,"450,00","450,00",0%,"90,00","90,00",Concluído,,,,SM-C108
1,08/07/2025 15:00,13/07/2025 11:00,5,5,Alex Palmer +55 63 99991 9458,Booking,"1.084,02",,"0,00","1.084,02",...,"249,20","216,80",13%,"49,84","43,36",Concluído,,"259,59",julho,SM-C108
2,13/07/2025 15:00,15/07/2025 11:00,2,2,‪Weiman Pereira +55 66 98448-1056‬,Airbnb,,,"400,96","400,96",...,"245,13","200,48",18%,"122,56","100,24",Concluído,,,,SM-C108
3,16/07/2025 15:00,18/07/2025 11:00,2,5,Paula Andrade +55 81 99716 3944,Booking,"436,74",,"0,00","436,74",...,"251,00","218,37",13%,"50,20","43,67",Concluído,,,,SM-C108
4,18/07/2025 15:00,20/07/2025 11:00,2,4,Cláudia +55 81 99974-5581‬,Airbnb,"290,00",,"292,39","582,39",...,"356,05","291,20",18%,"89,01","72,80",Concluído,,,,SM-C108


Criando o gráfico de ocupação...
Gráfico gerado.


Gráfico salvo como 'ocupacao_apartamentos.html'.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import plotly.graph_objects as go

# --- Função para verificar disponibilidade lógica (SEM ALTERAÇÕES) ---
def verificar_disponibilidade(df, data_inicio, data_fim):
    if df is None or df.empty: return [], []
    dt_inicio = pd.to_datetime(data_inicio)
    dt_fim = pd.to_datetime(data_fim)
    df_temp = df.copy()
    df_temp['Início'] = pd.to_datetime(df_temp['Início'], errors='coerce', dayfirst=True)
    df_temp['Fim'] = pd.to_datetime(df_temp['Fim'], errors='coerce', dayfirst=True)
    df_temp = df_temp.dropna(subset=['Início', 'Fim'])
    todos_aptos = sorted(df_temp['Apartamento'].unique().tolist())
    conflitos = df_temp[(df_temp['Início'] < dt_fim) & (df_temp['Fim'] > dt_inicio)]
    aptos_ocupados = sorted(list(set(conflitos['Apartamento'].unique())))
    aptos_livres = [ap for ap in todos_aptos if ap not in aptos_ocupados]
    return aptos_livres, aptos_ocupados

# --- Interface Gráfica (Widgets) ---
lbl_titulo = widgets.HTML("<h3>🔍 Consultar Disponibilidade</h3>")

# Widgets de Data
date_picker_inicio = widgets.DatePicker(description='Check-in', value=pd.to_datetime('today').date())
date_picker_fim = widgets.DatePicker(description='Check-out', value=(pd.to_datetime('today') + pd.Timedelta(days=1)).date())

btn_verificar = widgets.Button(description="Verificar Disponibilidade", button_style='primary', icon='search')
out_resultado = widgets.Output()

# --- LÓGICA NOVA: Sincronizar Datas ---
def atualizar_checkout(change):
    """
    Função chamada automaticamente quando o Check-in muda.
    Define o Check-out para (Check-in + 1 dia).
    """
    if change['new']: # Se houver uma nova data selecionada
        data_checkin = change['new']
        # Calcula dia seguinte
        nova_data_checkout = data_checkin + pd.Timedelta(days=1)
        # Atualiza o valor do widget de fim
        date_picker_fim.value = nova_data_checkout

# "Observa" mudanças no valor ('value') do date_picker_inicio
date_picker_inicio.observe(atualizar_checkout, names='value')
# --------------------------------------

# Função do Botão (Mantida igual à anterior visualmente)
def on_btn_click(b):
    with out_resultado:
        clear_output(wait=True)
        
        if not date_picker_inicio.value or not date_picker_fim.value:
            print("⚠️ Por favor, selecione ambas as datas.")
            return

        dt_ini = pd.to_datetime(date_picker_inicio.value)  + pd.Timedelta(hours=15)
        dt_fim = pd.to_datetime(date_picker_fim.value) + pd.Timedelta(hours=11)
        
        if dt_ini >= dt_fim:
            print("⚠️ ERRO: A data de Check-out deve ser posterior à de Check-in.")
            return

        # A. Calcular Disponibilidade
        livres, ocupados = verificar_disponibilidade(df_para_grafico, dt_ini, dt_fim)
        
        print(f"📅 Período: {dt_ini.strftime('%d/%m/%Y')} até {dt_fim.strftime('%d/%m/%Y')}")
        print("-" * 40)
        if livres:
            print(f"✅ DISPONÍVEIS ({len(livres)}): {', '.join(livres)}")
        else:
            print("❌ NENHUM APARTAMENTO DISPONÍVEL.")
        
        if ocupados:
            print(f"⛔ Ocupados ({len(ocupados)}): {', '.join(ocupados)}")
        print("-" * 40)

        # B. Gerar Gráfico
        fig_interativa = criar_grafico_ocupacao(df_para_grafico, exibir=False)
        
        if fig_interativa:
            # Barras de "Reserva Fantasma"
            if livres:
                hora_inicio_reserva = dt_ini# + pd.Timedelta(hours=15)
                hora_fim_reserva = dt_fim #+ pd.Timedelta(hours=11)
                duracao_ms = (hora_fim_reserva - hora_inicio_reserva).total_seconds() * 1000
                
                fig_interativa.add_trace(go.Bar(
                    name="Sua Seleção",
                    x=[duracao_ms] * len(livres),
                    y=livres,
                    base=[hora_inicio_reserva] * len(livres),
                    orientation='h',
                    marker=dict(color='rgba(255, 215, 0, 0.3)', line=dict(width=0)),
                    text=livres,
                    textposition='inside',
                    insidetextanchor='middle',
                    textfont=dict(color='black', size=12, weight='bold'),
                    hoverinfo="text",
                    hovertext=[f"<b>DISPONÍVEL: {ap}</b><br>Início: {hora_inicio_reserva.strftime('%d/%m %H:%M')}<br>Fim: {hora_fim_reserva.strftime('%d/%m %H:%M')}" for ap in livres],
                    offsetgroup="selecao" 
                ))

            # Ajustar Zoom
            fig_interativa.update_layout(
                xaxis=dict(range=[dt_ini - pd.Timedelta(days=3), dt_fim + pd.Timedelta(days=10)]),
                xaxis2=dict(range=[dt_ini - pd.Timedelta(days=3), dt_fim + pd.Timedelta(days=10)])
            )
            
            fig_interativa.show()

# Conectar evento e exibir
btn_verificar.on_click(on_btn_click)

ui = widgets.VBox([
    lbl_titulo,
    widgets.HBox([date_picker_inicio, date_picker_fim, btn_verificar]),
    out_resultado
])

display(ui)